<a href="https://colab.research.google.com/github/kartheekkotha/readBrainfMRI/blob/main/ML_brainData_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Installing requirements 

In [1]:
import pathlib
import numpy as np
from scipy.io import loadmat
import scipy.io
import os

In [4]:
import warnings

# Ignore warnings for collinearity and convergence
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [5]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.utils.validation import column_or_1d
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression, SGDClassifier
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import column_or_1d
import matplotlib.pyplot as plt

In [6]:
data_dir = 'datasets\ML_brainData'
desktop = pathlib.Path(data_dir)

In [7]:
dataset = []
for i in list(desktop.iterdir()):
    dataset.append(loadmat(i))

# Analysis for the first Person's fMRI 

## Data Pre-processing

### Getting the Initial Data

First let's create a np-array with 21764 features and 2 outputs condition and cond number or 1 output i.e word

In [8]:
mat_data = dataset[0]
mat_data_data =mat_data["data"]

In [9]:
nvoxels = mat_data["meta"][0][0]["nvoxels"][0][0]
print("The number of voxels are:",nvoxels)
ntrials = mat_data["meta"][0][0]["ntrials"][0][0]
print("The number of trials done are:",ntrials)

The number of voxels are: 21764
The number of trials done are: 360


In [10]:
data_firstPerson_withoutY = []
for i in range(0 , len(mat_data_data)):
  data_firstPerson_withoutY.append(mat_data_data[i][0][0])
print(len(data_firstPerson_withoutY[0]))

21764


In [11]:
data_firstPerson_withoutY = np.array(data_firstPerson_withoutY)
data_firstPerson_withoutY

array([[-0.0268068 ,  0.68483384,  0.9132587 , ..., -0.67066825,
        -1.45709228, -0.85008493],
       [ 0.91838692,  1.04450499,  0.77500801, ..., -1.82493971,
        -1.49682133, -1.52025864],
       [ 0.40446271,  0.49151523,  0.32273739, ..., -0.03602276,
         0.67378845, -0.55942959],
       ...,
       [-0.80578564, -0.80386184, -0.56857835, ...,  0.36546083,
         0.10383134,  0.97633791],
       [ 0.64031084,  0.57636939,  0.50777017, ...,  1.01490866,
         0.88595603,  0.39234203],
       [ 0.53591841,  0.83507612,  0.46409542, ...,  0.09621749,
        -0.17719461,  0.42357178]])

In [12]:
DF_withoutLabels = pd.DataFrame(data_firstPerson_withoutY)
# save the dataframe as a csv file without indexing
DF_withoutLabels.to_csv("data_firstPerson_withoutY.csv", index=False)
DF_withoutLabels

,0,1,2,3,4,5,6,7,8,9,...,21754,21755,21756,21757,21758,21759,21760,21761,21762,21763
0,-0.026807,0.684834,0.913259,-0.607473,-0.035652,0.831592,0.036146,0.103328,0.409886,0.377686,...,-1.659966,0.492116,2.379579,-0.373850,-1.888726,1.269766,0.644990,-0.670668,-1.457092,-0.850085
1,0.918387,1.044505,0.775008,0.446292,0.573288,0.296588,0.279037,0.688394,0.594526,0.285484,...,-2.874887,-1.267140,-0.787237,-1.540824,-1.145620,0.315128,-1.567767,-1.824940,-1.496821,-1.520259
2,0.404463,0.491515,0.322737,0.565998,0.910144,-0.022672,-0.265589,-0.320328,-0.144110,0.282993,...,-0.531833,-1.218505,-0.607947,-0.549644,-0.066790,-0.812939,-0.813185,-0.036023,0.673788,-0.559430
3,0.154143,0.090534,0.126065,0.851853,0.928996,0.568791,0.043317,0.174323,0.660360,0.907186,...,0.634760,0.154217,1.007080,-0.464818,-0.352520,1.451367,0.947464,0.376507,0.369274,1.379871
4,0.022657,0.066583,0.062565,-0.176771,-0.507001,0.426824,-0.443048,-0.417312,0.199288,0.620498,...,0.642981,1.752290,0.554075,2.382414,0.980933,1.072208,1.171389,0.802962,0.536364,0.772280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0.330510,0.082751,-0.564986,-0.783048,-0.649798,-0.519003,-0.144469,0.062276,-0.280306,-0.686704,...,0.085484,-0.369586,1.337798,0.595503,-0.037047,0.248788,0.387192,0.241775,-0.240881,0.073417
356,-0.437366,-0.199172,-0.021808,-0.100441,-0.047161,0.514779,-0.955829,-0.414140,0.144488,0.380802,...,-0.759127,-1.783479,-0.272532,-0.194764,-0.335785,0.563383,0.753249,0.595848,0.251088,-0.204628
357,-0.805786,-0.803862,-0.568578,-0.009557,0.061027,-0.602578,-0.333806,-0.546193,-1.111860,-1.289683,...,1.323663,2.428861,1.451445,1.555452,0.258751,1.229327,1.093605,0.365461,0.103831,0.976338
358,0.640311,0.576369,0.507770,0.785016,1.164965,0.152330,0.652013,0.236407,-0.028768,0.065255,...,0.412992,-0.305218,0.767470,-0.416772,-0.086066,0.390656,0.604469,1.014909,0.885956,0.392342


In [13]:
data_firstPerson_Y = []
resultData = mat_data["info"][0]
for i in range(0 , len(resultData)):
  word_no=(resultData[i]["cond_number"][0])
  data_firstPerson_Y.append(word_no)
data_firstPerson_Y = np.array(data_firstPerson_Y)
data_firstPerson_Y

array([[10],
       [ 4],
       [10],
       [10],
       [ 5],
       [11],
       [ 4],
       [ 7],
       [ 2],
       [ 9],
       [13],
       [ 8],
       [12],
       [ 3],
       [ 3],
       [ 7],
       [ 9],
       [13],
       [ 4],
       [ 6],
       [ 8],
       [11],
       [ 6],
       [ 5],
       [ 6],
       [ 4],
       [ 6],
       [12],
       [13],
       [13],
       [ 2],
       [12],
       [ 8],
       [ 6],
       [ 3],
       [ 5],
       [11],
       [ 9],
       [ 4],
       [ 9],
       [ 2],
       [ 9],
       [11],
       [ 2],
       [ 8],
       [12],
       [13],
       [ 3],
       [ 7],
       [11],
       [ 5],
       [ 3],
       [12],
       [10],
       [ 7],
       [ 5],
       [ 7],
       [ 8],
       [ 2],
       [10],
       [13],
       [ 3],
       [ 2],
       [13],
       [ 8],
       [11],
       [ 2],
       [ 3],
       [ 9],
       [11],
       [ 9],
       [ 4],
       [10],
       [ 4],
       [ 4],
       [11],
       [ 5],

In [14]:
data_firstPerson = np.append(data_firstPerson_withoutY, data_firstPerson_Y, axis=1)
DF_data_firstPerson = pd.DataFrame(data_firstPerson)

DF_data_firstPerson.to_csv("data_firstPerson_withY_withoutLabels.csv", index=False)
DF_data_firstPerson



,0,1,2,3,4,5,6,7,8,9,...,21755,21756,21757,21758,21759,21760,21761,21762,21763,21764
0,-0.026807,0.684834,0.913259,-0.607473,-0.035652,0.831592,0.036146,0.103328,0.409886,0.377686,...,0.492116,2.379579,-0.373850,-1.888726,1.269766,0.644990,-0.670668,-1.457092,-0.850085,10.0
1,0.918387,1.044505,0.775008,0.446292,0.573288,0.296588,0.279037,0.688394,0.594526,0.285484,...,-1.267140,-0.787237,-1.540824,-1.145620,0.315128,-1.567767,-1.824940,-1.496821,-1.520259,4.0
2,0.404463,0.491515,0.322737,0.565998,0.910144,-0.022672,-0.265589,-0.320328,-0.144110,0.282993,...,-1.218505,-0.607947,-0.549644,-0.066790,-0.812939,-0.813185,-0.036023,0.673788,-0.559430,10.0
3,0.154143,0.090534,0.126065,0.851853,0.928996,0.568791,0.043317,0.174323,0.660360,0.907186,...,0.154217,1.007080,-0.464818,-0.352520,1.451367,0.947464,0.376507,0.369274,1.379871,10.0
4,0.022657,0.066583,0.062565,-0.176771,-0.507001,0.426824,-0.443048,-0.417312,0.199288,0.620498,...,1.752290,0.554075,2.382414,0.980933,1.072208,1.171389,0.802962,0.536364,0.772280,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0.330510,0.082751,-0.564986,-0.783048,-0.649798,-0.519003,-0.144469,0.062276,-0.280306,-0.686704,...,-0.369586,1.337798,0.595503,-0.037047,0.248788,0.387192,0.241775,-0.240881,0.073417,5.0
356,-0.437366,-0.199172,-0.021808,-0.100441,-0.047161,0.514779,-0.955829,-0.414140,0.144488,0.380802,...,-1.783479,-0.272532,-0.194764,-0.335785,0.563383,0.753249,0.595848,0.251088,-0.204628,4.0
357,-0.805786,-0.803862,-0.568578,-0.009557,0.061027,-0.602578,-0.333806,-0.546193,-1.111860,-1.289683,...,2.428861,1.451445,1.555452,0.258751,1.229327,1.093605,0.365461,0.103831,0.976338,2.0
358,0.640311,0.576369,0.507770,0.785016,1.164965,0.152330,0.652013,0.236407,-0.028768,0.065255,...,-0.305218,0.767470,-0.416772,-0.086066,0.390656,0.604469,1.014909,0.885956,0.392342,11.0


In [15]:
print(DF_data_firstPerson.columns)

DF_data_firstPerson.rename(columns = {21764:'cond number'}, inplace = True)
# DF_data_firstPerson
print(DF_data_firstPerson.columns)

RangeIndex(start=0, stop=21765, step=1)
Index([            0,             1,             2,             3,
                   4,             5,             6,             7,
                   8,             9,
       ...
               21755,         21756,         21757,         21758,
               21759,         21760,         21761,         21762,
               21763, 'cond number'],
      dtype='object', length=21765)


### Labelling data according to the brain regions

- Functions to convert the data into MNI and label them according to the MNI coordinates
- Extracting the MNI coordinates and AAL labels for the data
- Mapping the data to the labels and grouping it

#### Functions to convert the data into MNI and label them according to the MNI coordinates

In [16]:
def get_mni_coordinates(v):
    """
    Get MNI coordinates for an array of N voxels.

    Parameters:
        v (ndarray): Nx3 array of VOXEL COORDINATES (voxel space).

    Returns:
        ndarray: Nx3 array of MNI points.
    """
    # Transformation matrix from vox coord to MNI coord
    transformToXYZmm = np.array(
        [[-3.125, 0, 0, 81.250], [0, 3.125, 0, -115.625], [0, 0, 6, -54.000]]
    )

    # Append a column of ones to v
    ones_column = np.ones((v.shape[0], 1))
    v_homogeneous = np.hstack((v, ones_column))

    # Perform the transformation
    p = np.dot(v_homogeneous, transformToXYZmm.T)[:, :3]

    return p


# Example usage:
voxel_coordinates = np.array([[45, 30, 12]])
mni_coordinates = get_mni_coordinates(voxel_coordinates)
print("MNI Coordinates:", mni_coordinates)

print("The shape of MNI coordinates are:", mni_coordinates.shape)   # 1x3

MNI Coordinates: [[-59.375 -21.875  18.   ]]
The shape of MNI coordinates are: (1, 3)


### Now we are going map the brain according to the regions of brains. Basically labelling where each voxel lies

In [17]:
file_dir = "datasets\ccbi_aal_atlas.mat"
atlas_data = scipy.io.loadmat(file_dir)
atlas_data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNX86, Created on: Tue Jan  6 13:15:54 2009',
 '__version__': '1.0',
 '__globals__': [],
 'Y': array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0,

In [18]:
label_data = atlas_data["label"]
print("The shape of label data is:", label_data.shape)   # 1x117
print("The length of label data is:", len(label_data[0]))   # 117 

The shape of label data is: (1, 117)
The length of label data is: 117


In [19]:
def get_aal_labels(p):
    """
    Get AAL labels for an array of N MNI points (mm).

    Parameters:
        p (ndarray): Nx3 array of MNI points.

    Returns:
        list: List of AAL labels.
    """
    # Load AAL atlas
    atlas_data = scipy.io.loadmat(file_dir)
    vinv = atlas_data["vinv"]
    Y = atlas_data["Y"]
    label = atlas_data["label"]
    id = atlas_data["id"]

    # Initialize variables
    pv = np.zeros((p.shape[0], 4))
    z = np.zeros(p.shape[0], dtype=int)
    L = []

    # Get labels
    for j in range(p.shape[0]):
        pv[j, :] = np.round(np.append(p[j, :], 1))
        pv[j, :] = np.dot(pv[j, :], vinv.T)
        z[j] = Y[int(pv[j, 0]), int(pv[j, 1]), int(pv[j, 2])]
        L.append(label[np.where(id == z[j])][0])

    return L


# Example usage:
mni_points = np.array([[-32, -42, -18], [16, -54, 14], [-40, -100, 30]])
aal_labels = get_aal_labels(mni_points)
print("AAL Labels:")
aal_labels

AAL Labels:


[array([[array(['Fusiform_L'], dtype='<U10')]], dtype=object),
 array([[array(['Precuneus_R'], dtype='<U11')]], dtype=object),
 array([[array(['Not_labelled'], dtype='<U12')]], dtype=object)]

#### Extracting the MNI coordinates and AAL labels for the data

In [20]:
coordinates = mat_data["meta"][0][0]["colToCoord"]
print("The total coordinates are:",len(coordinates))
print(coordinates[0:3])

The total coordinates are: 21764
[[31 10  1]
 [32 10  1]
 [33 10  1]]


In [21]:
MNI_coordinates = get_mni_coordinates(coordinates)
print("The total coordinates are:",len(MNI_coordinates))
print(MNI_coordinates[0:3])

The total coordinates are: 21764
[[-15.625 -84.375 -48.   ]
 [-18.75  -84.375 -48.   ]
 [-21.875 -84.375 -48.   ]]


In [22]:
AAL_labels = get_aal_labels(MNI_coordinates)
print("The total labels are:",len(AAL_labels))
AAL_labels[0:3]

The total labels are: 21764


[array([[array(['Not_labelled'], dtype='<U12')]], dtype=object),
 array([[array(['Cerebelum_Crus2_L'], dtype='<U17')]], dtype=object),
 array([[array(['Cerebelum_Crus2_L'], dtype='<U17')]], dtype=object)]

In [23]:
unique_labels = np.unique(AAL_labels)
print("Length of unique labels are : ", len(unique_labels))
print("The unique Labels are :")
for i, label in enumerate(unique_labels.tolist()):
    print(i, ") ", label[0], sep="")

Length of unique labels are :  117
The unique Labels are :
0) Amygdala_L
1) Amygdala_R
2) Angular_L
3) Angular_R
4) Calcarine_L
5) Calcarine_R
6) Caudate_L
7) Caudate_R
8) Cerebelum_10_L
9) Cerebelum_10_R
10) Cerebelum_3_L
11) Cerebelum_3_R
12) Cerebelum_4_5_L
13) Cerebelum_4_5_R
14) Cerebelum_6_L
15) Cerebelum_6_R
16) Cerebelum_7b_L
17) Cerebelum_7b_R
18) Cerebelum_8_L
19) Cerebelum_8_R
20) Cerebelum_9_L
21) Cerebelum_9_R
22) Cerebelum_Crus1_L
23) Cerebelum_Crus1_R
24) Cerebelum_Crus2_L
25) Cerebelum_Crus2_R
26) Cingulum_Ant_L
27) Cingulum_Ant_R
28) Cingulum_Mid_L
29) Cingulum_Mid_R
30) Cingulum_Post_L
31) Cingulum_Post_R
32) Cuneus_L
33) Cuneus_R
34) Frontal_Inf_Oper_L
35) Frontal_Inf_Oper_R
36) Frontal_Inf_Orb_L
37) Frontal_Inf_Orb_R
38) Frontal_Inf_Tri_L
39) Frontal_Inf_Tri_R
40) Frontal_Med_Orb_L
41) Frontal_Med_Orb_R
42) Frontal_Mid_L
43) Frontal_Mid_Orb_L
44) Frontal_Mid_Orb_R
45) Frontal_Mid_R
46) Frontal_Sup_L
47) Frontal_Sup_Medial_L
48) Frontal_Sup_Medial_R
49) Frontal_Sup_O

In [24]:
## Making the string array from the output of the AAL label conversion function
AAL_labels[0][0][0][0]
AAL_labels_modified = []
count  = 0
for i in range(0,len(AAL_labels)):
  count+=1
  AAL_labels_modified.append(AAL_labels[i][0][0][0])

In [25]:
AAL_labels[:10]

[array([[array(['Not_labelled'], dtype='<U12')]], dtype=object),
 array([[array(['Cerebelum_Crus2_L'], dtype='<U17')]], dtype=object),
 array([[array(['Cerebelum_Crus2_L'], dtype='<U17')]], dtype=object),
 array([[array(['Cerebelum_Crus2_L'], dtype='<U17')]], dtype=object),
 array([[array(['Cerebelum_Crus2_L'], dtype='<U17')]], dtype=object),
 array([[array(['Not_labelled'], dtype='<U12')]], dtype=object),
 array([[array(['Not_labelled'], dtype='<U12')]], dtype=object),
 array([[array(['Not_labelled'], dtype='<U12')]], dtype=object),
 array([[array(['Cerebelum_Crus2_L'], dtype='<U17')]], dtype=object),
 array([[array(['Cerebelum_Crus2_L'], dtype='<U17')]], dtype=object)]

In [26]:
AAL_labels_modified[:10]

['Not_labelled',
 'Cerebelum_Crus2_L',
 'Cerebelum_Crus2_L',
 'Cerebelum_Crus2_L',
 'Cerebelum_Crus2_L',
 'Not_labelled',
 'Not_labelled',
 'Not_labelled',
 'Cerebelum_Crus2_L',
 'Cerebelum_Crus2_L']

#### Mapping the data to the labels and grouping it

In [27]:
# read the csv file into a dataframe
DF_withoutLabels = pd.read_csv("data_firstPerson_withoutY.csv")
# drop the index from the dataframe
DF_withoutLabels

,0,1,2,3,4,5,6,7,8,9,...,21754,21755,21756,21757,21758,21759,21760,21761,21762,21763
0,-0.026807,0.684834,0.913259,-0.607473,-0.035652,0.831592,0.036146,0.103328,0.409886,0.377686,...,-1.659966,0.492116,2.379579,-0.373850,-1.888726,1.269766,0.644990,-0.670668,-1.457092,-0.850085
1,0.918387,1.044505,0.775008,0.446292,0.573288,0.296588,0.279037,0.688394,0.594526,0.285484,...,-2.874887,-1.267140,-0.787237,-1.540824,-1.145620,0.315128,-1.567767,-1.824940,-1.496821,-1.520259
2,0.404463,0.491515,0.322737,0.565998,0.910144,-0.022672,-0.265589,-0.320328,-0.144110,0.282993,...,-0.531833,-1.218505,-0.607947,-0.549644,-0.066790,-0.812939,-0.813185,-0.036023,0.673788,-0.559430
3,0.154143,0.090534,0.126065,0.851853,0.928996,0.568791,0.043317,0.174323,0.660360,0.907186,...,0.634760,0.154217,1.007080,-0.464818,-0.352520,1.451367,0.947464,0.376507,0.369274,1.379871
4,0.022657,0.066583,0.062565,-0.176771,-0.507001,0.426824,-0.443048,-0.417312,0.199288,0.620498,...,0.642981,1.752290,0.554075,2.382414,0.980933,1.072208,1.171389,0.802962,0.536364,0.772280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0.330510,0.082751,-0.564986,-0.783048,-0.649798,-0.519003,-0.144469,0.062276,-0.280306,-0.686704,...,0.085484,-0.369586,1.337798,0.595503,-0.037047,0.248788,0.387192,0.241775,-0.240881,0.073417
356,-0.437366,-0.199172,-0.021808,-0.100441,-0.047161,0.514779,-0.955829,-0.414140,0.144488,0.380802,...,-0.759127,-1.783479,-0.272532,-0.194764,-0.335785,0.563383,0.753249,0.595848,0.251088,-0.204628
357,-0.805786,-0.803862,-0.568578,-0.009557,0.061027,-0.602578,-0.333806,-0.546193,-1.111860,-1.289683,...,1.323663,2.428861,1.451445,1.555452,0.258751,1.229327,1.093605,0.365461,0.103831,0.976338
358,0.640311,0.576369,0.507770,0.785016,1.164965,0.152330,0.652013,0.236407,-0.028768,0.065255,...,0.412992,-0.305218,0.767470,-0.416772,-0.086066,0.390656,0.604469,1.014909,0.885956,0.392342


In [28]:
print(len(AAL_labels_modified))
print(len(DF_withoutLabels.columns))

21764
21764


In [29]:
# Add AAL_labels as a new row
DF_withoutLabels.loc["labels"] = AAL_labels_modified
DF_withoutLabels

,0,1,2,3,4,5,6,7,8,9,...,21754,21755,21756,21757,21758,21759,21760,21761,21762,21763
0,-0.026807,0.684834,0.913259,-0.607473,-0.035652,0.831592,0.036146,0.103328,0.409886,0.377686,...,-1.659966,0.492116,2.379579,-0.37385,-1.888726,1.269766,0.64499,-0.670668,-1.457092,-0.850085
1,0.918387,1.044505,0.775008,0.446292,0.573288,0.296588,0.279037,0.688394,0.594526,0.285484,...,-2.874887,-1.26714,-0.787237,-1.540824,-1.14562,0.315128,-1.567767,-1.82494,-1.496821,-1.520259
2,0.404463,0.491515,0.322737,0.565998,0.910144,-0.022672,-0.265589,-0.320328,-0.14411,0.282993,...,-0.531833,-1.218505,-0.607947,-0.549644,-0.06679,-0.812939,-0.813185,-0.036023,0.673788,-0.55943
3,0.154143,0.090534,0.126065,0.851853,0.928996,0.568791,0.043317,0.174323,0.66036,0.907186,...,0.63476,0.154217,1.00708,-0.464818,-0.35252,1.451367,0.947464,0.376507,0.369274,1.379871
4,0.022657,0.066583,0.062565,-0.176771,-0.507001,0.426824,-0.443048,-0.417312,0.199288,0.620498,...,0.642981,1.75229,0.554075,2.382414,0.980933,1.072208,1.171389,0.802962,0.536364,0.77228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,-0.437366,-0.199172,-0.021808,-0.100441,-0.047161,0.514779,-0.955829,-0.41414,0.144488,0.380802,...,-0.759127,-1.783479,-0.272532,-0.194764,-0.335785,0.563383,0.753249,0.595848,0.251088,-0.204628
357,-0.805786,-0.803862,-0.568578,-0.009557,0.061027,-0.602578,-0.333806,-0.546193,-1.11186,-1.289683,...,1.323663,2.428861,1.451445,1.555452,0.258751,1.229327,1.093605,0.365461,0.103831,0.976338
358,0.640311,0.576369,0.50777,0.785016,1.164965,0.15233,0.652013,0.236407,-0.028768,0.065255,...,0.412992,-0.305218,0.76747,-0.416772,-0.086066,0.390656,0.604469,1.014909,0.885956,0.392342
359,0.535918,0.835076,0.464095,-0.263199,0.344567,-0.221691,-0.618091,-0.176464,0.361088,0.613485,...,-0.281576,-0.958057,0.741373,-0.814091,-0.195442,0.325111,0.334351,0.096217,-0.177195,0.423572


In [30]:
DF_withoutLabels.to_csv("data_firstPerson_withoutY_withLabels_ungrouped.csv", index=False)

In [31]:
labels_row = DF_withoutLabels.loc['labels']
# Group the DataFrame by the 'labels' row and take the mean
grouped_df_withLabels = DF_withoutLabels.groupby(labels_row, axis=1).mean()
grouped_df_withLabels

labels,Amygdala_L,Amygdala_R,Angular_L,Angular_R,Calcarine_L,Calcarine_R,Caudate_L,Caudate_R,Cerebelum_10_L,Cerebelum_10_R,...,Thalamus_L,Thalamus_R,Vermis_10,Vermis_1_2,Vermis_3,Vermis_4_5,Vermis_6,Vermis_7,Vermis_8,Vermis_9
0,-0.012100,0.127622,0.317062,0.440933,0.982531,1.007493,0.527289,0.697380,-0.452706,0.112530,...,-0.039782,0.046021,-0.402737,-0.304329,0.108593,0.041929,0.218866,0.192530,-0.166507,-0.141913
1,0.184056,-0.252383,0.495670,0.268853,0.541290,0.385210,0.151791,0.061421,-0.137234,-0.151990,...,0.044482,-0.282987,-0.918588,-0.040708,-0.029827,0.101779,0.155766,0.424487,0.118813,0.009467
2,-0.026768,0.080071,0.282150,0.123677,0.316287,0.473524,0.191545,0.268181,0.215842,-0.007734,...,0.229800,0.086662,-0.084191,-0.005997,0.025533,-0.004611,0.339622,0.349004,0.021322,-0.097248
3,0.349658,0.278891,-0.038872,0.051120,0.468914,0.533926,0.302625,0.638368,0.128229,-0.021630,...,0.160389,-0.017670,-0.656971,-0.307707,0.235982,0.038905,0.133213,0.675041,-0.030475,-0.393777
4,0.071451,-0.138541,-0.290449,-0.082056,0.861988,0.837882,0.260126,0.170274,0.251325,0.313872,...,0.324789,0.184883,0.290392,0.159835,0.244467,0.026842,0.410277,0.282093,0.330347,0.219398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,-0.304918,-0.216347,-0.391553,-0.328098,0.760020,0.707223,-0.013529,-0.088941,0.072223,-0.223912,...,0.017763,-0.026142,-0.034494,-0.155840,-0.143455,0.115667,-0.010817,0.069590,-0.169807,0.428491
356,-0.604440,-0.640764,-0.290656,-0.194727,0.587302,0.777338,0.063159,0.167635,0.098304,0.018715,...,-0.173167,-0.049110,0.114216,-0.222488,-0.106257,-0.120006,0.073656,0.133038,-0.247422,0.121031
357,-0.339885,-0.027879,-0.226727,-0.315013,0.485950,0.449820,-0.047313,-0.207899,-0.243778,-0.083369,...,0.049297,-0.222177,-0.701982,-0.453526,-0.204876,0.111536,0.302387,0.631713,-0.141436,-0.227079
358,-0.087675,0.015724,-0.320493,-0.096281,0.305148,0.373391,0.648025,0.656058,0.018138,0.073058,...,-0.055691,-0.190539,-0.058743,0.152624,-0.512153,-0.132577,0.211198,0.468037,-0.055674,-0.345644


In [32]:
# save the data to csv file
grouped_df_withLabels.to_csv("data_firstPerson_withoutY_withLabels_grouped.csv")


## Research Paper Approach:

In [33]:
import string
import re
from decimal import *
words = ["airplane","ant","apartment","arch","arm","barn","bear","bed","bee","beetle","bell","bicycle","bottle","butterfly","car","carrot","cat","celery","chair","chimney","chisel","church","closet","coat","corn","cow","cup","desk","dog","door","dress","dresser","eye","fly","foot","glass","hammer","hand","horse","house","igloo","key","knife","leg","lettuce","pants","pliers","refrigerator","saw","screwdriver","shirt","skirt","spoon","table","telephone","tomato","train","truck","watch","window"]

# semantics_data[:] = semantics_data[:].apply(lambda s: s.astype(str))
feature_names=["see", "hear", "listen", "taste", "smell", "eat","touch", "rub", "lift", "manipulate", "run", "push","fill", "move", "ride", "say", "fear", "open", "approach", "near", "enter", "drive", "wear", "break","clean"]
semantics_data = pd.read_csv("python_data.csv", header=None)
index_list_rows = list(range(0,60))
index_list_columns = list(range(0,26))
index_list_columns_empty = list(range(0,25))
semantics_data.index = index_list_rows
semantics_data.columns = index_list_columns


empty_df = pd.DataFrame(index=semantics_data.index, columns=index_list_columns_empty)
for i in index_list_columns_empty:
  empty_df.iat[2,i]=0.0

for i in index_list_rows:
  for j in index_list_columns:
    if(j == 0): 
      continue

    elif(i==2 and j==25):
      continue
  
    else:  
      cell = semantics_data.iat[i,j]
      cell_index = cell.find("(")
      cell_name = cell[:cell_index]
      cell_name = cell_name.strip()
      cell_value = cell[cell_index+1:cell_index+6]
      cell_value = float(cell_value)
      cell_index_to_place = feature_names.index(cell_name)
      empty_df.iloc[i,cell_index_to_place] = cell_value





In [34]:
semantics_data.drop(semantics_data.iloc[:,1:], inplace=True, axis=1)
semantics_data
resultingsemanticdata = pd.concat([semantics_data, empty_df], axis=1)
# feature_names.insert(0,"stimulus words")
result_column_header = ["stimulus words"] + feature_names
resultingsemanticdata.columns = result_column_header

In [35]:
words_vector = resultingsemanticdata["stimulus words"]
resultingsemanticdata = resultingsemanticdata.drop(columns=["stimulus words"])

In [36]:

resultingsemanticdata = (resultingsemanticdata.div(resultingsemanticdata.sum(axis=1)/1000, axis=0)).round(2)
resultingsemanticdata.insert(0,"stimulus words",words_vector)

In [37]:
word_number = []

for i,word in enumerate (resultingsemanticdata["stimulus words"]):
    
    word_number.append((int)(i//5+2))
resultingsemanticdata["cond number"] = word_number

## Attempt to use wordvectors as per Harish sir's suggestion

In [38]:
import gensim
import gensim.downloader

vectors_25 = gensim.downloader.load('glove-twitter-25')
vectors_25.save("glove-twitter-25")

In [39]:
word_vector25_table = pd.DataFrame(columns = list(range(0,25)))
word_vector25_table["stimulus words"] = words_vector
for i,word in enumerate(words_vector):
    resultvector = vectors_25.get_vector(word)
    for j,value in enumerate(resultvector):
        word_vector25_table.loc[i][j] = value

In [40]:
resultingsemanticdata = word_vector25_table
word_number = []

for i,word in enumerate (resultingsemanticdata["stimulus words"]):
    
    word_number.append((int)(i//5+2))
resultingsemanticdata["cond number"] = word_number

In [41]:
classifier_data = resultingsemanticdata.drop(columns=["stimulus words"])

In [42]:
semantics_reference_table = resultingsemanticdata
word_number_list = resultingsemanticdata["cond number"]
bigtable_rows_index = list(range(0,360))
Y_semantics = pd.DataFrame(columns=resultingsemanticdata.columns, index=bigtable_rows_index)

In [43]:
semantics_reference_table.drop(columns=["stimulus words"], inplace=True)

In [44]:
semantics_reference_table = semantics_reference_table.groupby('cond number').transform('mean').drop_duplicates().reset_index().drop(columns=["index"])
semantics_reference_table["cond number"] = list(range(2,14))
semantics_reference_table

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,cond number
0,-1.045526,-0.459576,0.419970,0.446379,0.196816,-0.018738,1.155968,-0.290854,0.425668,0.410149,...,0.883984,-0.605410,0.079664,0.080589,0.067068,0.968562,0.189095,-0.538122,0.303440,2
1,-0.515930,-0.504880,-0.038342,0.351452,0.238869,0.351319,0.272484,-0.716256,-0.016532,0.645258,...,0.236722,0.073774,-0.721414,-0.194272,-0.791350,1.041630,1.474476,0.030170,-0.383604,3
2,-1.591920,0.059827,0.573590,-0.254669,-0.039772,-0.000836,0.224796,-1.102582,0.729456,0.243245,...,0.779592,0.116876,-0.149917,0.030420,-0.126342,-0.193872,-0.173316,-0.590162,0.801276,4
3,-1.213408,-0.042406,0.590666,0.190143,0.032074,-0.002962,0.017429,-0.988152,0.333238,0.453386,...,0.571156,0.457668,0.173428,0.027353,-0.409076,-0.093687,1.155812,-0.323000,0.072521,5
4,-0.934146,-1.173810,0.526364,-0.883272,0.297752,0.413016,0.147787,-1.031916,-0.228904,0.102389,...,0.632458,-1.274672,-0.418611,0.733488,-0.754770,0.763530,0.890950,0.213046,-0.658084,6
5,-1.386006,-0.319478,0.709488,-0.066215,0.650504,0.163304,0.088158,-1.265198,0.543522,0.437742,...,0.624230,-0.020973,-0.689450,-0.083066,-0.361011,-0.375459,0.859982,0.070951,-0.292996,7
6,-0.603294,-0.002020,0.390516,0.082422,-0.355484,0.191075,0.293966,-0.227251,0.580654,0.134247,...,0.518272,-0.150539,0.384068,0.574196,-0.095736,0.576066,0.613768,-0.157808,0.267380,8
7,-0.869696,0.061898,0.253050,0.652172,-0.052875,0.243549,0.392760,-0.927202,0.618866,0.278908,...,0.923853,-0.183614,0.286102,0.448686,-1.102542,0.399988,0.896992,0.283320,0.431846,9
8,-0.193654,0.102848,0.110358,0.037947,0.077530,-0.269679,0.581089,-0.460956,1.049482,0.063638,...,0.309016,0.158542,-0.040129,0.359847,-0.360309,-0.233185,0.557068,-0.046586,-0.320060,10
9,-0.553302,-0.266158,0.351855,0.311075,-0.059708,0.176634,-0.132757,-0.932236,0.784506,-0.260042,...,0.901452,0.235287,0.652316,-0.047158,-0.317320,0.517380,0.746734,0.109808,-0.410832,11


In [45]:

data_firstpersonY=list(data_firstPerson_Y)
for i,word_number in enumerate (data_firstPerson_Y):
    # word_number = .iloc[i]["cond number"]
    for j in list(range(0,12)):
        if(word_number == semantics_reference_table.loc[j]["cond number"]):
            Y_semantics.iloc[i,0:25] = semantics_reference_table.iloc[j,0:25]

Y_values = Y_semantics
Y_values.drop(columns=["cond number", "stimulus words"], inplace=True)

In [46]:
Total_Y_semantics = pd.DataFrame(columns=resultingsemanticdata.columns, index=list(range(0,3240)))
Total_Data_Y = pd.read_csv("merged_df_y.csv")
Total_Data_Y = Total_Data_Y.drop(columns=["Unnamed: 0"])
# Total_Data_Y=list(Total_Data_Y)
for i,word_number in enumerate (Total_Data_Y["cond_number"]):
    # word_number = .iloc[i]["cond number"]
    for j in list(range(0,12)):
        if(word_number == semantics_reference_table.loc[j]["cond number"]):
            Total_Y_semantics.iloc[i,0:25] = semantics_reference_table.iloc[j,0:25]

Total_Data_Y= Total_Y_semantics
Total_Data_Y.drop(columns=["cond number"], inplace=True)

### Linear regression model for 21,764 voxels' unlabeled data vs word vector data of length 25

In [60]:
Data_withoutLabels = pd.read_csv("data_firstPerson_withoutY.csv")
Data_withoutLabels

,0,1,2,3,4,5,6,7,8,9,...,21754,21755,21756,21757,21758,21759,21760,21761,21762,21763
0,-0.026807,0.684834,0.913259,-0.607473,-0.035652,0.831592,0.036146,0.103328,0.409886,0.377686,...,-1.659966,0.492116,2.379579,-0.373850,-1.888726,1.269766,0.644990,-0.670668,-1.457092,-0.850085
1,0.918387,1.044505,0.775008,0.446292,0.573288,0.296588,0.279037,0.688394,0.594526,0.285484,...,-2.874887,-1.267140,-0.787237,-1.540824,-1.145620,0.315128,-1.567767,-1.824940,-1.496821,-1.520259
2,0.404463,0.491515,0.322737,0.565998,0.910144,-0.022672,-0.265589,-0.320328,-0.144110,0.282993,...,-0.531833,-1.218505,-0.607947,-0.549644,-0.066790,-0.812939,-0.813185,-0.036023,0.673788,-0.559430
3,0.154143,0.090534,0.126065,0.851853,0.928996,0.568791,0.043317,0.174323,0.660360,0.907186,...,0.634760,0.154217,1.007080,-0.464818,-0.352520,1.451367,0.947464,0.376507,0.369274,1.379871
4,0.022657,0.066583,0.062565,-0.176771,-0.507001,0.426824,-0.443048,-0.417312,0.199288,0.620498,...,0.642981,1.752290,0.554075,2.382414,0.980933,1.072208,1.171389,0.802962,0.536364,0.772280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0.330510,0.082751,-0.564986,-0.783048,-0.649798,-0.519003,-0.144469,0.062276,-0.280306,-0.686704,...,0.085484,-0.369586,1.337798,0.595503,-0.037047,0.248788,0.387192,0.241775,-0.240881,0.073417
356,-0.437366,-0.199172,-0.021808,-0.100441,-0.047161,0.514779,-0.955829,-0.414140,0.144488,0.380802,...,-0.759127,-1.783479,-0.272532,-0.194764,-0.335785,0.563383,0.753249,0.595848,0.251088,-0.204628
357,-0.805786,-0.803862,-0.568578,-0.009557,0.061027,-0.602578,-0.333806,-0.546193,-1.111860,-1.289683,...,1.323663,2.428861,1.451445,1.555452,0.258751,1.229327,1.093605,0.365461,0.103831,0.976338
358,0.640311,0.576369,0.507770,0.785016,1.164965,0.152330,0.652013,0.236407,-0.028768,0.065255,...,0.412992,-0.305218,0.767470,-0.416772,-0.086066,0.390656,0.604469,1.014909,0.885956,0.392342


In [61]:
import numpy as np
from sklearn.model_selection import LeavePOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score

In [62]:
model = LinearRegression()
print(Data_withoutLabels.shape)
print(Y_values.shape)
X_train, X_test, y_train, y_test = train_test_split(Data_withoutLabels , Y_values, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(360, 21764)
(360, 25)
(288, 21764) (288, 25)
(72, 21764) (72, 25)


In [63]:
from sklearn.metrics import mean_squared_error, r2_score
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)
# Print evaluation metrics
print("Mean Squared Error:", mse)
print("R-squared:", r2)
pred_label = model.predict(Data_withoutLabels.iloc[1:2, :])

Mean Squared Error: 0.11243608599503388
R-squared: 0.24409523900011695


### Tried cross-validation as per Professor's recommendation:

In [64]:
from sklearn.model_selection import cross_val_score

# Define the number of folds
k = 10

# Perform k-fold cross-validation
scores = cross_val_score(model, X_train, y_train, cv=k, scoring='r2')

# Print the cross-validation scores
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

Cross-validation scores: [0.21997217 0.24635491 0.25225834 0.22453719 0.24949219 0.2392854
 0.20996424 0.10791203 0.23850313 0.20007045]
Mean cross-validation score: 0.21883500618601315


### Linear regression model with 117 labelled data feaatures

In [65]:
Data_withLabels = pd.read_csv("data_firstPerson_withoutY_withLabels_grouped.csv")
Data_withLabels = Data_withLabels.drop(columns=["Unnamed: 0"])
Data_withLabels

,Amygdala_L,Amygdala_R,Angular_L,Angular_R,Calcarine_L,Calcarine_R,Caudate_L,Caudate_R,Cerebelum_10_L,Cerebelum_10_R,...,Thalamus_L,Thalamus_R,Vermis_10,Vermis_1_2,Vermis_3,Vermis_4_5,Vermis_6,Vermis_7,Vermis_8,Vermis_9
0,-0.012100,0.127622,0.317062,0.440933,0.982531,1.007493,0.527289,0.697380,-0.452706,0.112530,...,-0.039782,0.046021,-0.402737,-0.304329,0.108593,0.041929,0.218866,0.192530,-0.166507,-0.141913
1,0.184056,-0.252383,0.495670,0.268853,0.541290,0.385210,0.151791,0.061421,-0.137234,-0.151990,...,0.044482,-0.282987,-0.918588,-0.040708,-0.029827,0.101779,0.155766,0.424487,0.118813,0.009467
2,-0.026768,0.080071,0.282150,0.123677,0.316287,0.473524,0.191545,0.268181,0.215842,-0.007734,...,0.229800,0.086662,-0.084191,-0.005997,0.025533,-0.004611,0.339622,0.349004,0.021322,-0.097248
3,0.349658,0.278891,-0.038872,0.051120,0.468914,0.533926,0.302625,0.638368,0.128229,-0.021630,...,0.160389,-0.017670,-0.656971,-0.307707,0.235982,0.038905,0.133213,0.675041,-0.030475,-0.393777
4,0.071451,-0.138541,-0.290449,-0.082056,0.861988,0.837882,0.260126,0.170274,0.251325,0.313872,...,0.324789,0.184883,0.290392,0.159835,0.244467,0.026842,0.410277,0.282093,0.330347,0.219398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,-0.304918,-0.216347,-0.391553,-0.328098,0.760020,0.707223,-0.013529,-0.088941,0.072223,-0.223912,...,0.017763,-0.026142,-0.034494,-0.155840,-0.143455,0.115667,-0.010817,0.069590,-0.169807,0.428491
356,-0.604440,-0.640764,-0.290656,-0.194727,0.587302,0.777338,0.063159,0.167635,0.098304,0.018715,...,-0.173167,-0.049110,0.114216,-0.222488,-0.106257,-0.120006,0.073656,0.133038,-0.247422,0.121031
357,-0.339885,-0.027879,-0.226727,-0.315013,0.485950,0.449820,-0.047313,-0.207899,-0.243778,-0.083369,...,0.049297,-0.222177,-0.701982,-0.453526,-0.204876,0.111536,0.302387,0.631713,-0.141436,-0.227079
358,-0.087675,0.015724,-0.320493,-0.096281,0.305148,0.373391,0.648025,0.656058,0.018138,0.073058,...,-0.055691,-0.190539,-0.058743,0.152624,-0.512153,-0.132577,0.211198,0.468037,-0.055674,-0.345644


In [66]:
model = LinearRegression()
print(Data_withLabels.shape)
print(Y_values.shape)
X_train, X_test, y_train, y_test = train_test_split(Data_withLabels , Y_values, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(360, 117)
(360, 25)
(288, 117) (288, 25)
(72, 117) (72, 25)


In [67]:
from sklearn.metrics import mean_squared_error, r2_score
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)
train_score = r2_score(y_train, y_train_pred)
print("Training score: ", train_score)
# Print evaluation metrics
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Training score:  0.5793945740353202
Mean Squared Error: 0.19094004575180284
R-squared: -0.2790291561060737


## Linear Regresion model using combined data of all nine people, grouped by AAL Labels with word vector - size 25

In [89]:
Total_Data_withLabels = pd.read_csv("merged_df_x.csv")
Total_Data_withLabels = Total_Data_withLabels.drop(columns=["Unnamed: 0"])
Total_Data_withLabels

,Amygdala_L,Amygdala_R,Angular_L,Angular_R,Calcarine_L,Calcarine_R,Caudate_L,Caudate_R,Cerebelum_10_L,Cerebelum_10_R,...,Thalamus_L,Thalamus_R,Vermis_10,Vermis_1_2,Vermis_3,Vermis_4_5,Vermis_6,Vermis_7,Vermis_8,Vermis_9
0,-0.012100,0.127622,0.317062,0.440933,0.982531,1.007493,0.527289,0.697380,-0.452706,0.112530,...,-0.039782,0.046021,-0.402737,-0.304329,0.108593,0.041929,0.218866,0.192530,-0.166507,-0.141913
1,0.184056,-0.252383,0.495670,0.268853,0.541290,0.385210,0.151791,0.061421,-0.137234,-0.151990,...,0.044482,-0.282987,-0.918588,-0.040708,-0.029827,0.101779,0.155766,0.424487,0.118813,0.009467
2,-0.026768,0.080071,0.282150,0.123677,0.316287,0.473524,0.191545,0.268181,0.215842,-0.007734,...,0.229800,0.086662,-0.084191,-0.005997,0.025533,-0.004611,0.339622,0.349004,0.021322,-0.097248
3,0.349658,0.278891,-0.038872,0.051120,0.468914,0.533926,0.302625,0.638368,0.128229,-0.021630,...,0.160389,-0.017670,-0.656971,-0.307707,0.235982,0.038905,0.133213,0.675041,-0.030475,-0.393777
4,0.071451,-0.138541,-0.290449,-0.082056,0.861988,0.837882,0.260126,0.170274,0.251325,0.313872,...,0.324789,0.184883,0.290392,0.159835,0.244467,0.026842,0.410277,0.282093,0.330347,0.219398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3235,0.589885,-0.098256,0.164627,-0.126710,-0.668116,-0.587220,0.786275,0.629815,0.000000,-1.124987,...,0.811939,0.705916,-0.555501,-0.231762,0.534972,0.179561,0.438018,-0.066340,0.119944,-0.602115
3236,-0.947762,0.078449,-0.781387,-0.593179,-1.021776,-0.786649,-1.040937,-1.082501,0.000000,0.127819,...,-0.536765,-0.588264,0.293013,-0.416266,-0.833310,-0.915601,-0.601780,-0.399891,-0.596439,-0.687656
3237,0.891211,0.885160,1.018131,1.172105,1.165504,1.054089,0.573844,0.343005,0.000000,-1.146360,...,0.351170,0.135977,-1.832284,-0.619526,-0.053624,0.613278,0.597363,0.278562,0.253301,-0.709114
3238,-0.518630,-0.951836,-0.579670,-0.520721,-0.753192,-0.687729,0.007167,-0.112834,0.000000,0.468434,...,-0.260458,-0.516816,0.684397,0.774259,-0.224130,-0.581611,-0.170178,-0.241677,-0.390929,-0.080807


In [90]:
model = LinearRegression()
print(Total_Data_withLabels.shape)
print(Total_Data_Y.shape)
X_train, X_test, y_train, y_test = train_test_split(Total_Data_withLabels , Total_Data_Y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(3240, 117)
(3240, 25)
(2592, 117) (2592, 25)
(648, 117) (648, 25)


In [91]:
from sklearn.metrics import mean_squared_error, r2_score
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)
# Print evaluation metrics
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.15333545590807734
R-squared: 0.03421102442715389


#### Used LeavePout cross-validation but as it took >4hours of run time, used a break statement to stop the iterations.

In [92]:
import numpy as np
from sklearn.model_selection import LeavePOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
model = LinearRegression()

l2o = LeavePOut(250)

r2_list = []
accuracy_score_list = []

X = Total_Data_withLabels
y = Total_Data_Y

for train_index, test_index in l2o.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test)

    # Calculate the mean squared error and append to the list
    r2 = r2_score(y_test, y_pred)
    print("r2 = ", r2)
    r2_list.append(r2)
    if (len(r2_list) == 100):
        break
    # Calculate the average mean squared error
average_r2 = np.mean(r2_list)

print("Average Mean Squared Error:", average_r2)

r2 =  0.09281458634718198


r2 =  0.09348582085315671
r2 =  0.09120981614111418
r2 =  0.09092009578708879
r2 =  0.09146587298265917
r2 =  0.09126180673128997
r2 =  0.09440852776500791
r2 =  0.09228670854291365
r2 =  0.09224501810537201
r2 =  0.09346166934367556
r2 =  0.0916602478661115
r2 =  0.09354107436745691
r2 =  0.09287719354508683
r2 =  0.09202068092634373
r2 =  0.09191567670007358
r2 =  0.09264555650868703
r2 =  0.09208557489232326
r2 =  0.0894225493584441
r2 =  0.09158761445858624
r2 =  0.0924688083040846
r2 =  0.08772873161775953
r2 =  0.09289601745609097
r2 =  0.09209250548548699
r2 =  0.09278968697281094
r2 =  0.09225881103799459
r2 =  0.09342397305544296
r2 =  0.09179637088202583
r2 =  0.09055487596045114
r2 =  0.09233163159414211
r2 =  0.09281446444022944
r2 =  0.0928732877002544
r2 =  0.09336867259235157
r2 =  0.09332451099554241
r2 =  0.09343809471323387
r2 =  0.09128807264094307
r2 =  0.09125274263446174
r2 =  0.09283055336162097
r2 =  0.0916376288614594
r2 =  0.08910163623863547
r2 =  0.093469263